In [1]:
%matplotlib inline
import numpy
import matplotlib.pyplot as plt
import scipy.io
data = scipy.io.loadmat('MSdata.mat')
import random
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

C:\Users\chevalier\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#split data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data["trainx"],data["trainy"], test_size =0.2, random_state = 30 )

In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
#from sklearn.svm import SVR
estimator = LinearRegression()
selector = RFE(estimator, 10, step=8)
selector = selector.fit(X_train, y_train)

X_train = selector.transform(X_train)
X_test = selector.transform(X_test)

C:\Users\chevalier\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [4]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
X_train_s = sc.fit_transform(X_train) 
X_test_s = sc.fit_transform(X_test)
y_train_s = sc.fit_transform(y_train)
y_test_s = sc.fit_transform(y_test)

C:\Users\chevalier\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype uint16 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [5]:
X_train_s = X_train_s.reshape((X_train_s.shape[0], 1, X_train_s.shape[1]))

#X_train_s = numpy.reshape(X_train_s, (X_train_s.shape[0], X_train_s.shape[1], 1))

In [6]:
X_test_s = X_test_s.reshape((X_test_s.shape[0], 1, X_test_s.shape[1]))

print(X_train_s.shape, y_train_s.shape, X_test_s.shape, y_test_s.shape)

(370972, 1, 10) (370972, 1) (92743, 1, 10) (92743, 1)


In [35]:
regressor = Sequential()

regressor.add(LSTM(units = 20, return_sequences = True, input_shape=(X_train_s.shape[1], X_train_s.shape[2])))
regressor.add(Dropout(0.2))
#regressor.add(LSTM(50, input_shape=(X_train_s.shape[1], X_train_s.shape[2])))
regressor.add(LSTM(units = 20, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 20, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 20, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 20))
regressor.add(Dropout(0.2))

regressor.add(Dense(1))
#regressor.compile(loss='mae', optimizer='adam')
regressor.compile(loss='mean_squared_error', optimizer='adam')

regressor.fit(X_train_s, y_train_s, epochs = 80, batch_size = 72, validation_data=(X_test_s, y_test_s), verbose=2, shuffle=False)

Train on 370972 samples, validate on 92743 samples
Epoch 1/80
 - 29s - loss: 0.0266 - val_loss: 0.0134
Epoch 2/80
 - 25s - loss: 0.0128 - val_loss: 0.0150
Epoch 3/80
 - 25s - loss: 0.0124 - val_loss: 0.0155
Epoch 4/80
 - 25s - loss: 0.0123 - val_loss: 0.0154
Epoch 5/80
 - 25s - loss: 0.0123 - val_loss: 0.0152
Epoch 6/80
 - 25s - loss: 0.0122 - val_loss: 0.0158
Epoch 7/80
 - 25s - loss: 0.0122 - val_loss: 0.0155
Epoch 8/80
 - 25s - loss: 0.0121 - val_loss: 0.0157
Epoch 9/80
 - 25s - loss: 0.0121 - val_loss: 0.0153
Epoch 10/80
 - 25s - loss: 0.0121 - val_loss: 0.0154
Epoch 11/80
 - 25s - loss: 0.0120 - val_loss: 0.0154
Epoch 12/80
 - 25s - loss: 0.0120 - val_loss: 0.0158
Epoch 13/80
 - 25s - loss: 0.0120 - val_loss: 0.0160
Epoch 14/80
 - 25s - loss: 0.0120 - val_loss: 0.0157
Epoch 15/80
 - 25s - loss: 0.0120 - val_loss: 0.0154
Epoch 16/80
 - 25s - loss: 0.0120 - val_loss: 0.0161
Epoch 17/80
 - 25s - loss: 0.0120 - val_loss: 0.0153
Epoch 18/80
 - 25s - loss: 0.0120 - val_loss: 0.0153
Epoc

In [36]:
y_pred = regressor.predict(X_test_s)
y_pred = sc.inverse_transform(y_pred)
y_pred

array([[1997.8086],
       [1998.1937],
       [1996.303 ],
       ...,
       [1999.5562],
       [1997.3315],
       [1994.0178]], dtype=float32)

In [37]:
y_test

array([[2010],
       [1982],
       [1967],
       ...,
       [2003],
       [1970],
       [2005]], dtype=uint16)

In [38]:
def test_linear(y_pred, y_test):
    n = len(y_pred)
    total = 0
    for i in range(n):
        total += abs(y_pred[i]-y_test[i][0])
    mae = total/n
    return mae    

well = test_linear(y_pred, y_test)
print(well)

[7.96209148]
